In [ ]:
# Core libraries
import os
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

# PyTorch and torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision import models, transforms, datasets
from torchvision.io import read_image

In [ ]:
!unzip .\facial-keypoint-detection\data\test.zip

In [ ]:
!unzip .\facial-keypoint-detection\data\training.zip

In [ ]:
class FacialKeypointsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.keypoints_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.keypoints_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.keypoints_frame.iloc[idx, 0])
        image = Image.open(img_name).convert("L")
        keypoints = self.keypoints_frame.iloc[idx, 1:].values.astype('float').reshape(-1, 2)

        if self.transform:
            image = self.transform(image)

        return image, keypoints

In [ ]:
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])